### 데이터 전처리 및 특징추출
- 캐글 파일

In [1]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os

In [6]:
m_files = []
directories = []
for i in os.walk('./data/genres(new_n)/'):
    for j in i:
        if isinstance(j, list) and j!=[]:
            m_files.append(j)
        elif j:
            directories.append(j)
wave_files = dict(zip(directories[1:], m_files[1:]))

In [7]:
m_files.remove(['.DS_Store'])

In [8]:
m_files

[['new'], ['country.wav', 'fusion.01.wav', 'fusion.00.wav', 'hiphop.wav']]

In [9]:
directories

['./data/genres(new_n)/', './data/genres(new_n)/new']

In [10]:
len(wave_files)

1

In [11]:
wave_files = dict(zip(directories[1:], m_files[1:]))

In [12]:
wave_files

{'./data/genres(new_n)/new': ['country.wav',
  'fusion.01.wav',
  'fusion.00.wav',
  'hiphop.wav']}

In [13]:
read_wave = {}
j = 0
for folder, file in wave_files.items():
    i = 0
    for read in file:
        read_wave[m_files[0][j]+str(i)]=os.path.join(folder, read)
        i+=1
    j+=1

In [14]:
len(read_wave)

4

In [15]:
def load_wav(files):
    all_loads = {}
    for folder, file in files.items():
        y, sr = librosa.load(file, sr=22050)
        all_loads[folder]=y
    return all_loads

In [16]:
loading_files = load_wav(read_wave)
loading_files

{'new0': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 'new1': array([ 7.44216720e-07,  1.24115366e-04, -1.20477176e-04, ...,
         5.28440952e-01,  5.83550632e-01,  6.29093230e-01], dtype=float32),
 'new2': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 'new3': array([0.        , 0.        , 0.        , ..., 0.44247332, 0.43032894,
        0.42878398], dtype=float32)}

In [17]:
# 데이터별 특징추출
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [18]:
datas = []
Y = []
for i, i1 in wave_files.items():
    for j in i1:
        data = os.path.join(i, j)
        y, sr = librosa.load(data)
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
        rmse = np.mean(librosa.feature.rms(y=y))
        spec_cent = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y))
        datas.append({
        "path":data,
        "filename":j,
        "chroma_stft":chroma_stft,
        "rmse":rmse,
        "spec_cent":spec_cent,
        "spec_bw":spec_bw,
        "rolloff":rolloff,
        "zcr":zcr
        })
        Y.append(i)

In [19]:
df = pd.DataFrame(datas)
df

,path,filename,chroma_stft,rmse,spec_cent,spec_bw,rolloff,zcr
0,./data/genres(new_n)/new/country.wav,country.wav,0.405833,0.150884,1801.780279,2061.809925,3808.076928,0.082311
1,./data/genres(new_n)/new/fusion.01.wav,fusion.01.wav,0.403036,0.201435,2612.195398,2712.788073,5855.150097,0.108375
2,./data/genres(new_n)/new/fusion.00.wav,fusion.00.wav,0.243923,0.157501,1261.581792,1530.421159,2268.777103,0.063182
3,./data/genres(new_n)/new/hiphop.wav,hiphop.wav,0.333927,0.193895,1922.524472,1913.085393,3828.246460,0.085060


In [20]:
head_mfcc = []
for i in range(1, 21):
    head_mfcc.append(f'mfcc{i}')

In [21]:
head_mfcc

['mfcc1',
 'mfcc2',
 'mfcc3',
 'mfcc4',
 'mfcc5',
 'mfcc6',
 'mfcc7',
 'mfcc8',
 'mfcc9',
 'mfcc10',
 'mfcc11',
 'mfcc12',
 'mfcc13',
 'mfcc14',
 'mfcc15',
 'mfcc16',
 'mfcc17',
 'mfcc18',
 'mfcc19',
 'mfcc20']

In [22]:
datas_mfcc = []
Y = []
for i, i1 in wave_files.items():
    for j in i1:
        data = os.path.join(i, j)
        y, sr = librosa.load(data)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        mfccs = {}
        for e in range(0,len(mfcc)):
            mfccs[head_mfcc[e]]=np.mean(mfcc[e])
        datas_tmp={
        "path":data,
        "filename":j
        }
        datas_tmp = dict(datas_tmp, **mfccs)
        datas_mfcc.append(datas_tmp)
        Y.append(i)

In [23]:
df_mfcc = pd.DataFrame(datas_mfcc)
df_mfcc

,path,filename,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,./data/genres(new_n)/new/country.wav,country.wav,-138.416382,107.744438,-8.081799,35.410576,16.586304,17.696692,-0.815842,2.527678,...,-5.747252,4.911643,-1.861773,3.432274,-3.833768,3.443918,-3.195200,4.867384,-2.693485,-1.800244
1,./data/genres(new_n)/new/fusion.01.wav,fusion.01.wav,-59.468258,77.297852,7.072240,31.523220,4.327747,4.600645,-5.130556,5.345969,...,-9.011683,6.837155,-8.763196,0.700869,-7.219707,3.109256,-12.741076,0.762420,-12.925092,0.018498
2,./data/genres(new_n)/new/fusion.00.wav,fusion.00.wav,-162.999817,137.448135,-11.774125,13.389125,-5.358283,8.140820,-4.449211,-0.121845,...,-8.824728,-3.346426,-9.124072,-5.896582,-4.805422,-4.856084,-5.165923,-1.054640,-3.473535,3.911199
3,./data/genres(new_n)/new/hiphop.wav,hiphop.wav,-126.523468,104.139671,-25.170931,7.303225,-8.411119,-4.023900,-17.508011,-3.665806,...,-12.640080,-5.408213,-4.625746,6.598872,-2.563092,6.465259,-0.746290,3.566419,-3.256784,-1.105662


In [24]:
df_all = pd.merge(df,df_mfcc)
df_all

,path,filename,chroma_stft,rmse,spec_cent,spec_bw,rolloff,zcr,mfcc1,mfcc2,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,./data/genres(new_n)/new/country.wav,country.wav,0.405833,0.150884,1801.780279,2061.809925,3808.076928,0.082311,-138.416382,107.744438,...,-5.747252,4.911643,-1.861773,3.432274,-3.833768,3.443918,-3.195200,4.867384,-2.693485,-1.800244
1,./data/genres(new_n)/new/fusion.01.wav,fusion.01.wav,0.403036,0.201435,2612.195398,2712.788073,5855.150097,0.108375,-59.468258,77.297852,...,-9.011683,6.837155,-8.763196,0.700869,-7.219707,3.109256,-12.741076,0.762420,-12.925092,0.018498
2,./data/genres(new_n)/new/fusion.00.wav,fusion.00.wav,0.243923,0.157501,1261.581792,1530.421159,2268.777103,0.063182,-162.999817,137.448135,...,-8.824728,-3.346426,-9.124072,-5.896582,-4.805422,-4.856084,-5.165923,-1.054640,-3.473535,3.911199
3,./data/genres(new_n)/new/hiphop.wav,hiphop.wav,0.333927,0.193895,1922.524472,1913.085393,3828.246460,0.085060,-126.523468,104.139671,...,-12.640080,-5.408213,-4.625746,6.598872,-2.563092,6.465259,-0.746290,3.566419,-3.256784,-1.105662


In [25]:
labels = df_all['filename'].str.split('.')
labels

0       [country, wav]
1    [fusion, 01, wav]
2    [fusion, 00, wav]
3        [hiphop, wav]
Name: filename, dtype: object

In [30]:
df_all['label'] = labels.str.get(0)

In [31]:
df_all['label'].unique(),len(df_all['label'].unique())

(array(['country', 'fusion', 'hiphop'], dtype=object), 3)

In [32]:
df_all.to_csv("./data/only_new_n_file.csv",index=False)

In [34]:
# 데이터 불러오기
df2 = pd.read_csv("./data/only_new_n_file.csv")
df2.head()

,path,filename,chroma_stft,rmse,spec_cent,spec_bw,rolloff,zcr,mfcc1,mfcc2,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,./data/genres(new_n)/new/country.wav,country.wav,0.405833,0.150884,1801.780279,2061.809925,3808.076928,0.082311,-138.416380,107.74444,...,4.911643,-1.861773,3.432274,-3.833768,3.443918,-3.195200,4.867384,-2.693485,-1.800244,country
1,./data/genres(new_n)/new/fusion.01.wav,fusion.01.wav,0.403036,0.201435,2612.195398,2712.788073,5855.150097,0.108375,-59.468258,77.29785,...,6.837155,-8.763196,0.700869,-7.219707,3.109256,-12.741076,0.762420,-12.925092,0.018498,fusion
2,./data/genres(new_n)/new/fusion.00.wav,fusion.00.wav,0.243923,0.157501,1261.581792,1530.421159,2268.777103,0.063182,-162.999820,137.44814,...,-3.346426,-9.124072,-5.896582,-4.805422,-4.856084,-5.165923,-1.054640,-3.473535,3.911199,fusion
3,./data/genres(new_n)/new/hiphop.wav,hiphop.wav,0.333927,0.193895,1922.524472,1913.085393,3828.246460,0.085060,-126.523470,104.13967,...,-5.408213,-4.625746,6.598872,-2.563093,6.465259,-0.746290,3.566419,-3.256784,-1.105662,hiphop
